# Garimpagem de Dados

## Aula 4 - Exercídio de Classificação com kNN

13/10/2017

**Dataset:** Titanic: Machine Learning from Disaster

https://www.kaggle.com/c/titanic/data

Partindo da aula passada:

1. Atualizar a função que mede a distância euclidiana para o pacote do scikit-learn 

2. Implementar uma função que selecione os k vizinhos mais próximos (k > 1)

3. Implementar uma função que recebe os k vizinhos mais próximos e determinar a classe correta

4. Transformar as features categoricas em numéricas (tip: pandas ou scikit-learn)

5. Analisar a necessidade de normalizar as features numéricas (tip: pandas ou scikit-learn)

6. Selecionar as features baseada na correlação (tip: pandas)

7. Separar o dataset em treino (75%) / teste (25%) / validação (10% do treino)

4. Execute o classificador para 30 k's pulando de 4 em 4 e apresente todas as acurácias utilizando o dataset de validação (Qual o melhor k?) [plotar um gráfico com os resultados]

5. Executar o classificador para o melhor k encontrado utilizando o dataset de teste e apresentar um relatório da precisão (tip: scikit-learn) [plotar um gráfico com os resultados]

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
class KNNClassifier(object):
    def __init__(self):
        self.X_train = None
        self.y_train = None

    def euc_distance(self, a, b):
        return np.linalg.norm(a-b)

    def closest(self, row):
        dists = [self.euc_distance(row, item) for  item in self.X_train]
        nei = dists.index(min(dists))
        
        return self.y_train[nei]
    
    def k_closests(self, row, n_of_neighbors):
        distance_array = []
        k_nearest_array = []
        for i in self.X_train:
            distance_array.append(self.euc_distance(i, row))
            
        for i in range(0, n_of_neighbors):
            k_nearest_array.append(distance_array.index(min(distance_array)))
            del distance_array[distance_array.index(min(distance_array))]

    def fit(self, training_data, training_labels, k=1, distance='euclidean'):
        self.X_train = training_data
        self.y_train = training_labels
        self.k = k
        self.distance = distance

    def predict(self, to_classify):
        predictions = []
        for row in to_classify:
            label = self.closest(row)
            predictions.append(label)
        return predictions

In [ ]:
d = pd.read_csv('titanic.csv')
d.head()

In [ ]:
d.describe()

In [ ]:
del d['Cabin']
del d['Ticket']
del d['PassengerId']
del d['Name']
d.head()

In [ ]:
d['Age'] = d.Age.fillna(d.Age.mean()) #média das idades, pra onde tem nada.
d = d.where((pd.notnull(d)), 0)
for f in ["Sex", "Embarked"]:
    d[f] = d[f].astype('category')
    d[f] = d[f].cat.codes

d.head()

In [ ]:
sex_encoder = LabelEncoder().fit(d.Sex)

d['Sex_encoded'] = sex_encoder.transform(d.Sex)

embarked_encoder = LabelEncoder().fit(d.Embarked.fillna("None"))

d["Embarked_encoded"] = embarked_encoder.transform(d.Embarked.fillna("None"))

In [ ]:
pvt=['Pclass', 'Sex_encoded', 'Embarked_encoded', 'Age', 'SibSp', 'Parch']


In [ ]:
d[pvt].corr()

In [ ]:
x = MinMaxScaler().fit_transform(d[pvt])

y = d.Survived.values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25)

In [ ]:
knn = KNNClassifier()
knn.fit(x_train, y_train, k=10, distance='euclidean')
result = knn.predict(x_test)
score = metrics.accuracy_score(y_pred = result, y_true = y_test)
print(score)